In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [2]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
api_base = os.environ["GRAPHRAG_API_BASE"]
api_version = os.environ["GRAPHRAG_API_VERSION"]

llm = ChatOpenAI(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [3]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../graphfleet/output/20240827-012901/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Total report count: 239
Report count after filtering by community level 2: 185


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,237,# AGIEval and AI Model Performance Evaluation\...,3,8.5,AGIEval and AI Model Performance Evaluation,The rating is high due to AGIEval's significan...,"The community revolves around AGIEval, a human...",[{'explanation': 'AGIEval is a pivotal benchma...,"{\n ""title"": ""AGIEval and AI Model Performa...",c32ecac8-6dea-4fd4-817d-1cd6518f98de
1,238,# Orca-3 and its Ecosystem\n\nThe community re...,3,9.0,Orca-3 and its Ecosystem,The report is highly significant due to the de...,The community revolves around the Orca-3 langu...,[{'explanation': 'Orca-3 has shown notable enh...,"{\n ""title"": ""Orca-3 and its Ecosystem"",\n ...",99902b15-234c-4514-b9db-e04c7ac0f595
2,134,# Agents and Reading Comprehension in AI\n\nTh...,2,8.5,Agents and Reading Comprehension in AI,The rating is high due to the significant impa...,The community focuses on the development and o...,[{'explanation': 'AGENT is a central entity in...,"{\n ""title"": ""Agents and Reading Comprehens...",86999f2f-567c-43ac-809d-d85f8df8092c
3,135,# Seed Instruction Generation Flow and Compreh...,2,8.5,Seed Instruction Generation Flow and Comprehen...,The rating is high due to the significant impa...,The community centers around the 'Seed Instruc...,[{'explanation': 'The Seed Instruction Generat...,"{\n ""title"": ""Seed Instruction Generation F...",73aeb516-a05a-4069-9bed-0f365a739667
4,136,# Instruction Refinement Flow in AgentInstruct...,2,8.5,Instruction Refinement Flow in AgentInstruct,The rating is high due to the significant impa...,The community centers around the Instruction R...,[{'explanation': 'The Instruction Refinement F...,"{\n ""title"": ""Instruction Refinement Flow i...",054de93a-962b-4a11-8c95-44ac2c350e15


#### Build global context based on community reports

In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [6]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [7]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=True,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [8]:
result = await search_engine.asearch(
    "Write an example of multi-agent system that incorporates the tree search algorithm? and GraphRAG?Show me an agent that can help me create a knowledge graph"
)

print(result.response)

### Multi-Agent System Incorporating Tree Search Algorithm and GraphRAG

A multi-agent system that incorporates both the tree search algorithm and GraphRAG can be designed to enhance decision-making and knowledge graph creation. This system leverages the strengths of Monte Carlo Tree Search (MCTS) for exploring decision paths and GraphRAG for organizing and retrieving information.

#### Language Agent Tree Search (LATS)

One example of such a system is the Language Agent Tree Search (LATS) framework. LATS integrates MCTS to enable language models to act as agents, facilitating proficient exploration and enhanced decision-making. The system uses MCTS to construct the best trajectory from sampled actions, making it a powerful tool for model-based reinforcement learning and decision-making environments. LATS has demonstrated high performance in tasks such as HotPotQA, HumanEval, and the Game of 24 [Data: Reports (62, 140, 156, 182, +more)].

### GraphRAG for Knowledge Graph Creation

Grap

In [9]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,40,Quality-Diversity and Lu's Contributions in AI...,0.464286,# Quality-Diversity and Lu's Contributions in ...,9.0
1,153,Tree-of-Thought Prompting and Search Algorithm...,0.214286,# Tree-of-Thought Prompting and Search Algorit...,9.0
2,168,LLM Debate and Key AI Techniques,0.214286,# LLM Debate and Key AI Techniques\n\nThe comm...,9.0
3,43,GSM-Hard Dataset and Key Contributors,0.214286,# GSM-Hard Dataset and Key Contributors\n\nThe...,8.5
4,81,Graph-Based RAG Applications and Key Technologies,0.142857,# Graph-Based RAG Applications and Key Technol...,9.0
...,...,...,...,...,...
180,75,MIRAGE Benchmark and Medical QA Datasets,0.107143,# MIRAGE Benchmark and Medical QA Datasets\n\n...,8.5
181,141,Zhuang and Search Algorithms in AI,0.107143,# Zhuang and Search Algorithms in AI\n\nThe co...,8.5
182,51,LM Agent and Sequential Decision-Making in AI,0.071429,# LM Agent and Sequential Decision-Making in A...,9.0
183,137,Suggester Agent and Genetic Predisposition in ...,0.035714,# Suggester Agent and Genetic Predisposition i...,8.5


In [10]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 13. LLM tokens: 148682
